<a href="https://colab.research.google.com/github/RaulMrSouza/Covid_19_Casos_Por_100k_habitantes_no_Estado_de_S-o_Paulo/blob/master/Covid_19_Casos_Por_100k_habitantes_no_Estado_de_S%C3%A3o_Paulo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [997]:
#pacotes e imports iniciais
!pip install folium==0.8.2
import folium
from folium import plugins
import pandas as pd
import numpy as np
import json
!pip install requests==v2.21.0
import requests

In [0]:
#trazer json de malha geografica da api do Ibge

headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}
cidades = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/35/?formato=application/vnd.geo+json&resolucao=5",
                           headers=headers)
cidades = cidades.json()


In [0]:
#baixar csv do dataset
url="https://brasil.io/dataset/covid19/caso_full/?state=SP&format=csv"
covid = pd.read_csv(url)
covid = covid[covid.is_last == True]
#desconsiderar estado
covid = covid[covid.city_ibge_code != 35]
#remover nulos
covid = covid[covid.last_available_confirmed_per_100k_inhabitants > 0]
covid = covid[~covid.last_available_confirmed.isnull()]

In [0]:
covid['codarea'] = covid['city_ibge_code']
covid = covid.set_index('codarea')

In [1001]:
covid = covid.sort_values(by=['last_available_confirmed_per_100k_inhabitants'], ascending=False)
covid[['city', 'last_available_confirmed_per_100k_inhabitants', 'last_available_confirmed', 'last_available_deaths', 'estimated_population_2019']].head(10)

,city,last_available_confirmed_per_100k_inhabitants,last_available_confirmed,last_available_deaths,estimated_population_2019
codarea,,,,,
3524501.0,Jaci,1103.72152,78,2,7067.0
3547650.0,Santa Salete,776.69903,12,0,1545.0
3548500.0,Santos,768.50115,3330,145,433311.0
3536208.0,Pariquera-Açu,508.95765,100,2,19648.0
3550308.0,São Paulo,478.44344,58619,4239,12252023.0
3505708.0,Barueri,422.71192,1159,126,274182.0
3520202.0,Igaratá,409.06230,39,0,9534.0
3534401.0,Osasco,383.29482,2677,302,698418.0
3547304.0,Santana de Parnaíba,344.93392,481,16,139447.0


In [1002]:
#mapa de cores de zero até maior caso
from branca.colormap import linear

colormap = linear.YlOrRd_09.scale(0, 
                   c['last_available_confirmed_per_100k_inhabitants'].max()
                   )
colormap

In [0]:
#adicionar campos para layout no mapa para cada cidade
for cidade in cidades['features']: 
  codarea = cidade['properties']['codarea']
  if int(codarea) in c.index.values:
    cidade['properties']['cidadeNome'] = str(covid.loc[int(codarea), "city"])
    cidade['properties']['last_available_confirmed_per_100k_inhabitants'] = int(covid.loc[int(codarea), "last_available_confirmed_per_100k_inhabitants"])
    cidade['properties']['style'] = {
        'fillColor': colormap(int(covid.loc[int(codarea), "last_available_confirmed_per_100k_inhabitants"])),
        'color': 'black',
        'weight': 0.8,
        'fill_opacity' : 1.0
    }
  else:
    cidade['properties']['cidadeNome'] = ''
    cidade['properties']['last_available_confirmed_per_100k_inhabitants'] = 0
    cidade['properties']['style'] = {
        'fillColor': 'grey',
        #'color': 'grey',
        'weight': 0.0,
        'fill_opacity' : 0.0
    }

In [1004]:
#mapa de São Paulo com casos por cidade
m = folium.Map(
    location=[-22,-48],zoom_start=7.3
)
folium.GeoJson(cidades).add_to(m)
folium.GeoJson(cidades).add_to(m)
folium.GeoJson(cidades).add_to(m)
colormap.caption = 'Casos Confirmados por 100 k habitantes'
folium.LayerControl().add_to(m)
m.add_child(colormap)
m